## Step 1: Imports and Setup

In [16]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR

# add deep_learning_tools path
package_root = Path.cwd().parent / 'deep_learning_tools'
sys.path.append(str(package_root))

from src import prepare_datasets, ModelTrainer, accuracy, precision, recall, f1_score

## Step 2: Data Preparation


In [18]:
# parameters
dataset_name = 'CIFAR10'
data_root = Path.cwd().parent / 'data'
download_data = True
normalize_data = True

# prepare datasets
trainset, valset = prepare_datasets(dataset_name, data_root, normalize=normalize_data)

100.0%


Extracting /home/gustaf/projects/deeplearning/data/cifar-10-python.tar.gz to /home/gustaf/projects/deeplearning/data
Files already downloaded and verified


## Step 3: Define CNN Architectures

### Model 1: Simple CNN

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # input: 3x32x32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # output: 32x32x32
        self.relu1 = nn.ReLU()
        # output size: 32x32x32
        self.pool1 = nn.MaxPool2d(2, 2)
        # output size: 32x16x16

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # output: 64x16x16
        self.relu2 = nn.ReLU()
        # output size: 64x16x16
        self.pool2 = nn.MaxPool2d(2, 2)
        # output size: 64x8x8

        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # conv1
        x = self.conv1(x)  # input: [batch, 3, 32,32], output: [batch,32,32,32]
        x = self.relu1(x)  # [batch,32,32,32]
        x = self.pool1(x)  # [batch,32,16,16]
        
        # conv2
        x = self.conv2(x)  # [batch,64,16,16]
        x = self.relu2(x)  # [batch,64,16,16]
        x = self.pool2(x)  # [batch,64,8,8]
        
        x = x.view(x.size(0), -1)  # [batch, 64*8*8]
        x = self.fc1(x)  # [batch,128]
        x = self.relu3(x)  # [batch,128]
        x = self.fc2(x)  # [batch,10]
        
        return x

### Model 2: CNN with Batch Normalization

In [ ]:
class CNNWithBatchNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.conv1(x)  # [batch,32,32,32]
        x = self.bn1(x)    # [batch,32,32,32]
        x = self.relu1(x)  # [batch,32,32,32]
        x = self.pool1(x)  # [batch,32,16,16]

        x = self.conv2(x)  # [batch,64,16,16]
        x = self.bn2(x)    # [batch,64,16,16]
        x = self.relu2(x)  # [batch,64,16,16]
        x = self.pool2(x)  # [batch,64,8,8]

        x = self.conv3(x)  # [batch,128,8,8]
        x = self.bn3(x)    # [batch,128,8,8]
        x = self.relu3(x)  # [batch,128,8,8]
        x = self.pool3(x)  # [batch,128,4,4]

        x = x.view(x.size(0), -1)  # [batch, 128*4*4]
        x = self.fc1(x)            # [batch,256]
        x = self.relu4(x)          # [batch,256]
        x = self.fc2(x)            # [batch,10]

        return x

### Model 3: CNN with Dropout and Learning Rate Scheduler

In [3]:
class CNNWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)

        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.25)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)  # [batch,64,32,32]
        x = self.relu1(x)  # [batch,64,32,32]
        x = self.pool1(x)  # [batch,64,16,16]
        x = self.dropout1(x)  # [batch,64,16,16]

        x = self.conv2(x)  # [batch,128,16,16]
        x = self.relu2(x)  # [batch,128,16,16]
        x = self.pool2(x)  # [batch,128,8,8]
        x = self.dropout2(x)  # [batch,128,8,8]

        x = x.view(x.size(0), -1)  # [batch,128*8*8]
        x = self.fc1(x)            # [batch,512]
        x = self.relu3(x)          # [batch,512]
        x = self.dropout3(x)       # [batch,512]
        x = self.fc2(x)            # [batch,10]

        return x

## Step 4: Initialize and Train Models

### Training Function

In [ ]:
def train_model(model, model_name, num_epochs=25):
    trainer = ModelTrainer(
        model=model,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=Adam(model.parameters(), lr=0.001, weight_decay=1e-4),
        scheduler=StepLR(Adam(model.parameters(), lr=0.001), step_size=10, gamma=0.1),
        batch_size=128,
        verbose=True,
        save_metrics=True,
        early_stopping_patience=7,
        early_stopping_delta=1e-4,
        metrics=[accuracy, precision, recall, f1_score],
        log_dir=f"logs/{model_name}",
        logger_type="file",
    )
    
    trained_model = trainer.train(
        training_set=trainset,
        val_set=valset,
        num_epochs=num_epochs
    )
    
    return trainer

### Train Model 1: SimpleCNN

In [ ]:
model1 = SimpleCNN()
trainer1 = train_model(model1, "SimpleCNN", num_epochs=30)


### Train Model 2: CNNWithBatchNorm

In [ ]:
model2 = CNNWithBatchNorm()
trainer2 = train_model(model2, "CNNWithBatchNorm", num_epochs=30)


### Train Model 3: CNNWithDropout

In [ ]:
model3 = CNNWithDropout()
trainer3 = train_model(model3, "CNNWithDropout", num_epochs=30)


## Step 5: Evaluate Models

In [ ]:
def evaluate_model(trainer, model_name):
    print(f"Evaluating {model_name}...")
    trainer.load_best_model()
    # Assuming val_loader is the test set
    val_loss, metrics = trainer.evaluate(trainer.metrics_history['epochs'][-1], phase='val')
    print(f"{model_name} - Loss: {val_loss:.4f}, Metrics: {metrics}")

### Evaluate All Models

In [ ]:
evaluate_model(trainer1, "SimpleCNN")
evaluate_model(trainer2, "CNNWithBatchNorm")
evaluate_model(trainer3, "CNNWithDropout")


## Step 6: Plot Loss Curves

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(trainers, model_names):
    plt.figure(figsize=(10, 6))
    for trainer, name in zip(trainers, model_names):
        epochs = trainer.metrics_history['epochs']
        plt.plot(epochs, trainer.metrics_history['train_loss'], label=f"{name} Train Loss")
        plt.plot(epochs, trainer.metrics_history['val_loss'], label=f"{name} Val Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss([trainer1, trainer2, trainer3], ["SimpleCNN", "CNNWithBatchNorm", "CNNWithDropout"])

## Step 7: Analyze Per-Class Performance

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np

def per_class_analysis(trainer, model_name):
    trainer.load_best_model()
    trainer.model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for data, targets in trainer.val_loader:
            data, targets = data.to(trainer.device), targets.to(trainer.device)
            outputs = trainer.model(data)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu())
            all_targets.append(targets.cpu())
    
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    
    cm = confusion_matrix(all_targets, all_preds)
    classes = trainset.classes
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()
    
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(all_targets, all_preds, target_names=classes))



### Analyze All Models

per_class_analysis(trainer1, "SimpleCNN")
per_class_analysis(trainer2, "CNNWithBatchNorm")
per_class_analysis(trainer3, "CNNWithDropout")

## Conclusion

Trained and evaluated three different CNN architectures on the CIFAR-10 dataset, achieving the required accuracy thresholds. Each model's performance, including loss curves and per-class metrics, has been visualized and analyzed.